<a href="https://colab.research.google.com/github/huluobohua/polycrisis/blob/main/Coding_data_transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://github.com/huluobohua/polycrisis/blob/main/0.%20Polycrisis%20dashboard%20-%20Raw%20data%20for%20Mariana2.xlsx?raw=true

# Coding data transformations

First, let's mount Google Drive so we can load files from Google Drive Folders.

In [ ]:
#/* from google.colab import drive
#drive.mount('/content/gdrive')
#%cd /content/gdrive/'Shared with me'/'Unicef Foresight and Policy' */

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: '/content/gdrive/Shared with me/Unicef Foresight and Policy/'
/content


Then, we'll load the file that contains the data to be dispayed.

In [1]:
#Locate file
import pandas as pd
import openpyxl
f = 'https://github.com/huluobohua/polycrisis/blob/main/0.%20Polycrisis%20dashboard%20-%20Raw%20data%20for%20Mariana2.xlsx?raw=true'


### GBP indicator B

In [2]:
#Load xlsx sheet
import numpy as np
import pandas as pd
df1 = pd.read_excel(f,
  sheet_name="GDP indicator B",
  skiprows=1,
  nrows=197,
  usecols="A:K"
);

df1.head()

/usr/local/lib/python3.8/dist-packages/pandas/io/excel/_base.py:1272: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return self._reader.parse(


,ISO Code,IMF growth forecast for 2023 in Oct 21,IMF growth forecast for 2023 in Oct 22,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme
0,ABW,6.200,1.950,ABW,Aruba,Latin America and the Caribbean,NaN,0,0
1,AFG,NaN,NaN,AFG,Afghanistan,Southern Asia,SA,0,1
2,AGO,3.328,3.427,AGO,Angola,Sub-Saharan Africa,ESA,0,1
3,AIA,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1
4,ALB,4.100,2.500,ALB,Albania,Southern Europe,ECA,0,1


In [4]:
#Replicate column C using A1 and A2
a1_gdpb = "IMF growth forecast for 2023 in Oct 21"
a2_gdpb = "IMF growth forecast for 2023 in Oct 22"


df1_c = ( df1[a2_gdpb] - df1[a1_gdpb] ) / abs(df1[a1_gdpb]) #operation
assert all( df1_c.dropna() == df1[c_gdpb].dropna() ) #sanity check

df1_c.head()

KeyError: ignored

In [ ]:
#Replicate column D using C
d_gdpb = "Relative IMF growth projection shifted right"

df1_d = df1_c + abs( df1_c.min() ) #operation
assert all( df1_d.dropna() == df1[d_gdpb].dropna() ) #sanity check

df1_d.head()

0    1.454516
1         NaN
2    2.169748
3    1.749756
4    1.880741
dtype: float64

In [ ]:
#Replicate column I using D
i_gdpb = "Normalized multiplied by 10 and direction shifted (higher is worse)"

#normalisation
q1, q3 = df1_d.quantile(.25), df1_d.quantile(.75)
outlier_uppercut = q3 + 2*(q3-q1)
filter = df1_d.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((d-df1_d.min())/(df1_d[filter].max()-df1_d.min()) if d < outlier_uppercut
     else (1. if not np.isnan(d) else np.nan))
     for d in df1_d 
])

#reindexing
df1_i = 10 - 10*(normalisation)
df1_i.head()

0    4.832758
1         NaN
2    2.291863
3    3.783903
4    3.318574
dtype: float64

### Food inflation indicator

In [ ]:
#Load xlsx sheet
df2 = pd.read_excel(f,
  sheet_name="Food inflation indicator",
  skiprows=1,
  nrows=196,
  usecols="A:G"
);

df2.head()

,Country,ISO,Food inflation,Food inflation shifted right,Normalized,ISO.1,Normalized*10
0,Aruba,ABW,11.53,12.51,0.255306,ABW,2.553061
1,Afghanistan,AFG,17.59,18.57,0.378980,AFG,3.789796
2,Angola,AGO,16.80,17.78,0.362857,AGO,3.628571
3,Albania,ALB,15.30,16.28,0.332245,ALB,3.322449
4,Andorra,AND,NaN,NaN,NaN,AND,NaN


In [ ]:
#Replicate column B using A
a_fii = "Food inflation"
b_fii = "Food inflation shifted right"

df2_b = df2[a_fii] + abs(df2[a_fii].min()) #operation
assert all( df2_b.dropna() == df2[b_fii].dropna() ) #sanity check

df2_b.head()

0    12.51
1    18.57
2    17.78
3    16.28
4      NaN
Name: Food inflation, dtype: float64

In [ ]:
#Replicate column I*10 using B
i_fii = "Normalized*10"

#normalisation
q1, q3 = df2_b.quantile(.25), df2_b.quantile(.75)
outlier_uppercut = q3 + 2*(q3-q1) 
#q3 + 2*(q3-q1) there is a manual override in the sheet
filter = df2_b.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((b-df2_b.min())/(df2_b[filter].max()-df2_b.min()) if b < outlier_uppercut
     else (1. if not np.isnan(b) else np.nan))
     for b in df2_b 
])

#reindexing
df2_i = 10*(normalisation)
df2_i

0      2.601913
1      3.862313
2      3.698003
3      3.386023
4           NaN
         ...   
191         NaN
192    9.993482
193    2.699667
194    7.850501
195    8.390537
Length: 196, dtype: float64

### Credit Score indicator

In [ ]:
#Load xlsx sheet
df3 = pd.read_excel(f,
  sheet_name="Credit Score indicator",
  skiprows=1,
  nrows=196,
  usecols="A:G"
);

df3.head()

,Country,ISO,Credit rating average,Credit rating average inverted (to make more positive worse),Credit rating average normalized,ISO.1,Credit rating average normalized * 10 (values in red are imputted - exclude from indexation range)
0,Aruba,ABW,11.5,9.5,0.452381,ABW,4.523810
1,Afghanistan,AFG,NaN,NaN,NaN,AFG,7.500000
2,Angola,AGO,6.0,15.0,0.714286,AGO,7.142857
3,Albania,ALB,8.0,13.0,0.619048,ALB,6.190476
4,Andorra,AND,14.0,7.0,0.333333,AND,3.333333


In [ ]:
#Replicate column B using A
a_csi = "Credit rating average"
b_csi = "Credit rating average inverted (to make more positive worse)"

df3_b = 21. - df3[a_csi] #operation
assert all( df3_b.dropna() == df3[b_csi].dropna() ) #sanity check

df3_b.head()

0     9.5
1     NaN
2    15.0
3    13.0
4     7.0
Name: Credit rating average, dtype: float64

In [ ]:
#Replicate column I*10 using B
i_csi = "Credit rating average normalized * 10 (values in red are imputted - exclude from indexation range)"

#normalisation
q1, q3 = df3[a_csi].quantile(.25), df3[a_csi].quantile(.75) #using column A
# q1, q3 = df3_b.quantile(.25), df3_b.quantile(.75) #using column B
outlier_uppercut = q3 + 2*(q3-q1)
filter = df3_b.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((b-df3_b.min())/(df3_b[filter].max()-df3_b.min()) if b < outlier_uppercut
     else (1. if not np.isnan(b) else np.nan))
     for b in df3_b 
])

#reindexing
df3_i = 10*(normalisation)
df3_i

0      4.523810
1           NaN
2      7.142857
3      6.190476
4      3.333333
         ...   
191         NaN
192         NaN
193    5.555556
194    9.047619
195    9.523810
Length: 196, dtype: float64

### External conflict risk

In [ ]:
#Load xlsx sheet
df4 = pd.read_excel(f,
  sheet_name="External conflict risk",
  skiprows=1,
  nrows=196,
  usecols="A:G"
);

df4.head()

In [ ]:
#Replicate column C123 using A1, A2  and A3
a1_ecr = "GPI Militarization 2022"
a2_ecr = "GPI Relationship with neighbors 2022"
a3_ecr = "Current external conflict (5 if current; 3 if probable or stand-off; 0 if none)"
c123_ecr = "GPI External Components"

df4_c123 = ( df4[a1_ecr]+df4[a2_ecr]+df4[a3_ecr] )/3 #operation
assert all( df4_c123.dropna() == df4[c123_ecr].dropna() ) #sanity check

df4_c123.head()

In [ ]:
#Replicate column I using C123
i_ecr = "GPI external components normalized"

#normalisation
q1, q3 = df4_c123.quantile(.25), df4_c123.quantile(.75) #using column C123
outlier_uppercut = q3 + 2*(q3-q1)
filter = df4_c123.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((c-df4_c123.min())/(df4_c123[filter].max()-df4_c123.min()) if c < outlier_uppercut
     else (1. if not np.isnan(c) else np.nan))
     for c in df4_c123 
])

#reindexing
df4_i = 10*(normalisation)
df4_i

### Political stability abs conf

In [ ]:
#Load xlsx sheet
df5 = pd.read_excel(f,
  sheet_name="Political stability abs conf",
  skiprows=1,
  nrows=196,
  usecols="A:D"
)

df5.head()

In [ ]:
#Replicate column I using A
a_psac = "2021 WGI Political stability"
i_psac = "WGI normalized"

#normalisation
q1, q3 = df5[a_psac].quantile(.25), df5[a_psac].quantile(.75) #using column A
outlier_uppercut = q3 + 2*(q3-q1)
filter = df5[a_psac].between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((a-df5[a_psac].min())/(df5[a_psac][filter].max()-df5[a_psac].min()) if a < outlier_uppercut
     else (1. if not np.isnan(a) else np.nan))
     for a in df5[a_psac]
])

#reindexing
df5_i = 10 - 10*(normalisation)
df5_i

### Climate indicator

In [ ]:
#Load xlsx sheet
df6 = pd.read_excel(f,
  sheet_name="Climate indicator",
  skiprows=1,
  nrows=196,
  usecols="A:E"
);

df6.head()

In [ ]:
#Replicate column I using A
a_ci = "ND-GAIN Climate Vulnerability Index"
i_ci = "Climate index taken to 10 point scale"

#normalisation
q1, q3 = df6[a_ci].quantile(.25), df6[a_ci].quantile(.75)
outlier_uppercut = q3 + 2*(q3-q1)
q1,q3,outlier_uppercut
filter = df6[a_ci].between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((a-df6[a_ci].min())/(df6[a_ci][filter].max()-df6[a_ci].min()) if a < outlier_uppercut
     else (1. if not np.isnan(a) else np.nan))
     for a in df6[a_ci]
])

#reindexing
df6_i = 10*(normalisation)
df6_i